# Step 3: Calculate OD

In [1]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

from shapely.geometry import LineString, Point

In [2]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

In [3]:
from GOSTnets.load_traffic2 import *

In [4]:
# read graph
G = nx.read_gpickle('./sri_lanka_unclean2_w_time_largest_20200616_traffic_mean_speeds.pickle')

In [5]:
#len(G.edges)

In [6]:
#gn.example_edge(G, 5)

## load origins

In [7]:
origins = gpd.read_file('./origins_destinations/intl_airport_updated.shp')

In [8]:
origins

,osm_id,emergencyh,aeroway,operatorty,source,capacitype,addrfull,emergency,name,addrcity,building,POINT_X,POINT_Y,geometry
0,183539977.0,None,None,None,None,None,None,None,Bandaranaike Int'l Airport,None,None,79.8846,7.1773,POINT (79.88658 7.17454)


## load destinations

In [9]:
destinations = gpd.read_file('./origins_destinations/cities_top10.shp')

In [10]:
destinations

,osm_id,code,fclass,population,name,country,geometry
0,44240758,1001,city,27500,Nuwara Eliya,LKA,POINT (80.76707 6.97382)
1,50794342,1005,national_capital,753000,Colombo,LKA,POINT (79.85385 6.93500)
2,258049423,1001,city,40800,Matale,LKA,POINT (80.62342 7.47213)
3,330173100,1001,city,45400,Puttalam,LKA,POINT (79.82866 8.03019)
4,335253618,1001,city,99100,Trincomalee,LKA,POINT (81.23450 8.57643)
5,2050947471,1001,city,600000,Jaffna,LKA,POINT (80.00930 9.66509)
6,2908772560,1001,city,125000,Kandy,LKA,POINT (80.63508 7.29309)
7,3289447451,1001,city,20000,Gampaha,LKA,POINT (79.99514 7.09256)
8,3639539242,1001,city,110000,Matara,LKA,POINT (80.54829 5.94782)
9,3639546887,1001,city,120000,Galle,LKA,POINT (80.21495 6.03281)


In [11]:
origins_gdf = gn.pandana_snap_c(G, origins, source_crs = 'epsg:4326', target_crs = 'epsg:32644')

In [12]:
#origins_gdf

In [13]:
origins_list = list(set(origins_gdf.NN))

In [14]:
destinations_gdf = gn.pandana_snap_c(G, destinations, source_crs = 'epsg:4326', target_crs = 'epsg:32644')

In [15]:
#destinations_gdf

In [16]:
destinations_list = list(set(destinations_gdf.NN))

In [17]:
#destinations_list

## Calculate OD

In [18]:
# It will use the default weight of 'time'
import time

start = time.time()

OD = gn.calculate_OD(G, origins_list, destinations_list, fail_value = 9999999)

end = time.time()
print(end - start)

6.517165184020996


In [19]:
#OD

In [20]:
OD_df = pd.DataFrame(OD, index = origins_list, columns = destinations_list)

In [21]:
OD_df

,4140985760,4784044133,1734550181,4004620073,3379679501,3289453904,1243386867,7098642100,1727494683,5385270047
3935302581,8870.548745,15949.046648,7956.761856,10012.53686,6971.13744,2290.562977,18187.297201,24519.939042,11282.291313,2028.452838


In [22]:
#OD_df.min(axis=0)

### Now we need to find the nearest city for each tourist point, and based on this assign a group of tourist points to each of the 10 cities

In [23]:
# test
# first column is the destination (tourist point), and the second column is the nearest city
OD_df.idxmin(axis=0)

4140985760    3935302581
4784044133    3935302581
1734550181    3935302581
4004620073    3935302581
3379679501    3935302581
3289453904    3935302581
1243386867    3935302581
7098642100    3935302581
1727494683    3935302581
5385270047    3935302581
dtype: int64

In [24]:
#OD_df.idxmin(axis=0).to_csv('./nodes_and_associated_nearest_city.csv')

In [25]:
#OD_df.idxmin(axis=0).to_frame(0)

### takes the min index value of each column, then groups by city (first index (0)) and takes the first entry

In [26]:
groupby_obj = OD_df.idxmin(axis=0).to_frame(0).groupby(0)[0]

In [27]:
groupby_obj

In [28]:
# visualize groupby_obj
#groupby_obj.apply(list)

In [29]:
#type(groupby_obj.apply(list))

In [30]:
# a nice way to visualize the groupby_obj
#groupby_obj.describe()

### create a dictionary that associates assigned tourist points with each city

In [31]:
city_tourist_pt_dict = {}
for name, group in groupby_obj:
    #print(group)
    for items in group.iteritems(): 
        #print(items[1])
        if items[1] not in city_tourist_pt_dict:
            city_tourist_pt_dict[items[1]] = [items[0]]
        else:
            #append value to list in dict value
            city_tourist_pt_dict[items[1]].append(items[0])
    #print(type(group))
    #print(group.head(1))

    #print(name)
    #print(city_tourist_pt_dict[group])

In [32]:
city_tourist_pt_dict

{3935302581: [4140985760,
  4784044133,
  1734550181,
  4004620073,
  3379679501,
  3289453904,
  1243386867,
  7098642100,
  1727494683,
  5385270047]}

## Loop through dictionary in order to do a calculate_OD for each airport's nearest cities

In [33]:
OD = {}
OD_df_dict = {}
for city,tourist_pts in city_tourist_pt_dict.items():
    OD[city] = gn.calculate_OD(G, [city], tourist_pts, fail_value = 9999999)
    OD_df_dict[city] = pd.DataFrame(OD[city], index = [city], columns = tourist_pts)

In [34]:
#sample_df = OD_df_dict[4784044133]
#sample_df

In [35]:
OD_df_dict

{3935302581:              4140985760    4784044133   1734550181   4004620073  3379679501  \
 3935302581  8870.548745  15949.046648  7956.761856  10012.53686  6971.13744   
 
              3289453904    1243386867    7098642100    1727494683   5385270047  
 3935302581  2290.562977  18187.297201  24519.939042  11282.291313  2028.452838  }

### time calculateOD time for airport city and its set of city points

In [36]:
#start = time.time()
#OD[4784044133] = gn.calculate_OD(G, [4784044133], tourist_pts, fail_value = 9999999)
#print(time.time() - start)

In [37]:
# sum of all shortest routes
#OD[4784044133][0].sum()

## Now work on generating routes and visualizing them

In [38]:
from shapely.ops import linemerge
from itertools import islice

In [39]:
maxspeed_mean_speeds = {
    'secondary': 50,
    'secondary_link': 45,
    'tertiary': 40,
    'tertiary_link': 40,
    'residential': 25,
    'unclassified': 25,
}

In [ ]:
mapbox_mean_speeds = {
    'secondary': 34,
    'secondary_link': 9,
    'tertiary': 25,
    'tertiary_link': 13,
    'residential': 20,
    'unclassified': 20,
}

In [40]:
def tabulate_edges(route):
    edge_table = []
    route_geometry = LineString()
    for idx in range(0, len(route) - 1):
        #edge_table.append([route[idx], route[idx+1]])
        # look up line
        #print('to node')
        #print(route[idx])
        #print('from node')
        #print(route[idx+1])
        edge_geometry = G.get_edge_data(route[idx],route[idx+1])[0]['geometry']
        # get edge speed
        edge_speed = G.get_edge_data(route[idx],route[idx+1])[0]['speed']
        #print('print edge_speed')
        #print(edge_speed)
        
        # compare edge speed to median speed
        rural_roads_list = ['residential','secondary','secondary_link','tertiary','tertiary_link','unclassified']
        
        edge_infra_type = G.get_edge_data(route[idx],route[idx+1])[0]['infra_type']
        
        edge_length = G.get_edge_data(route[idx],route[idx+1])[0]['length']
        edge_time = G.get_edge_data(route[idx],route[idx+1])[0]['time']
        
        mean_speed = G.get_edge_data(route[idx],route[idx+1])[0]['mean_speed']
        
        try:
            edge_imp_cost = G.get_edge_data(route[idx],route[idx+1])[0]['imp_cost']
        except:
            edge_imp_cost = 0
            pass
        
        if mean_speed > 0:      
            if edge_infra_type in rural_roads_list:
                print('print edge attributes')
                print(G.get_edge_data(route[idx],route[idx+1])[0])
                #print('print improved speed')
                #print(mapbox_traffic_mean_speeds[edge_infra_type])
                #assums that current edge lenght is in km
                new_time_s = (edge_length / mapbox_mean_speeds[edge_infra_type]) * 3600
                #new_time_s = ((edge_length / 1000) / maxspeed_mean_speeds[edge_infra_type]) * 3600
                #print("print new_time_s")
                #print(new_time_s)
                #print('print old time')
                #print(edge_time)
                edge_savings = edge_time - new_time_s
                # assign savings time
                edge_table.append([route[idx], route[idx+1], edge_savings, edge_imp_cost, edge_length, edge_infra_type, mean_speed, edge_geometry])
                # data['sec_saved'] = data['length'] / new_time
                # data['improvement_cost'] = 174861 * data['length'] / 1000
            else:
                # data['sec_saved'] = 0
                # very high number
                # data['improvement_cost'] = 10000000000
                #print("not a rural road")
                edge_table.append([route[idx], route[idx+1], 0, 0, edge_length, edge_infra_type, mean_speed, edge_geometry])
        else:
            edge_table.append([route[idx], route[idx+1], 0, 0, edge_length, edge_infra_type, mean_speed, edge_geometry])
            
        route_geometry = route_geometry.union(edge_geometry)
        
    #print('print route_geometry')
    #print(route_geometry)
    
    return(edge_table, route_geometry)

In [41]:
def generate_complete_edges_and_routes(input_df):

    LIMIT = 1000000

    complete_edges = []
    complete_routes = []

    count = 0

    # for origin, row in sample_df.iterrows(): 
    for origin, row in islice(input_df.iterrows(), LIMIT):    
        for destination, value in islice(row.items(), LIMIT):
            try:
                origin = int(origin)
            except:
                pass
            try:
                destination = int(destination)
            except:
                pass

            count = count + 1

            route = nx.dijkstra_path(G, origin, destination, weight = 'time')
            #path_edges = zip(route,route[1:])
            #print('print path_edges')
            #print(list(path_edges))
            # print(route)
            edge_table, route_geometry = tabulate_edges(route)
            #print('print edge_table:')
            #print(edge_table)
            complete_edges = complete_edges + edge_table
            #print('route_time')
            #print(value)
            complete_routes.append([edge_table[0][0], edge_table[-1][1], value, route_geometry])
            #print('edge_table[:-1]')
            #print(edge_table[-1][1])
            
    # convert complete_edges to gdf
    complete = pd.DataFrame(complete_edges, columns = ['o', 'd', 'sec_saved', 'imp_cost', 'length', 'infra_type', 'mean_speed', 'geometry'])
    complete['w'] = 1
    complete_count = complete.groupby(['o','d']).agg(
        {
            'w':"count",
            'sec_saved': 'first',
            'imp_cost': 'first',
            'mean_speed': 'first',
            'length':'first',
            'infra_type':'first',
            'geometry':'first'
        }
    )
    complete_count.reset_index(inplace = True)
    complete_count['o'] = complete_count['o'].astype(str)
    complete_count['d'] = complete_count['d'].astype(str)
    complete_count['weighted_sec_saved'] = complete_count.w * complete_count.sec_saved
    complete_count.sort_values(by=['weighted_sec_saved'], ascending=False)
    complete_count_gdf = gpd.GeoDataFrame(complete_count, crs = 'epsg:4326')
    
    # convert complete_routes to gdf
    complete_routes_df = pd.DataFrame(complete_routes, columns = ['origin','destination','time','geometry'])
    complete_routes_gdf = gpd.GeoDataFrame(complete_routes_df, crs = 'epsg:4326')
        
    return [complete_count_gdf, complete_routes_gdf]

In [42]:
#results = generate_complete_edges_and_routes(OD_df[4784044133])

In [43]:
OD_df_dict

{3935302581:              4140985760    4784044133   1734550181   4004620073  3379679501  \
 3935302581  8870.548745  15949.046648  7956.761856  10012.53686  6971.13744   
 
              3289453904    1243386867    7098642100    1727494683   5385270047  
 3935302581  2290.562977  18187.297201  24519.939042  11282.291313  2028.452838  }

In [44]:
for key in OD_df_dict:
    print(key)

3935302581


In [ ]:
results = generate_complete_edges_and_routes(OD_df_dict[3935302581])

print edge attributes
{'osm_id': 447572464, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f0076cbbc10>, 'infra_type': 'residential', 'min_speed': 1.0, 'max_speed': 26.0, 'mean_speed': 13.01984126984127, 'length': 0.013810216750416217, 'speed': 13.01984126984127, 'imp_cost': 2579.7484889777493, 'time': 3.818539663510391, 'mode': 'drive'}
print edge attributes
{'osm_id': 447572464, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f00648e2210>, 'infra_type': 'residential', 'min_speed': 9.0, 'max_speed': 30.0, 'mean_speed': 23.486607142857142, 'length': 0.04029001072972453, 'speed': 23.486607142857142, 'imp_cost': 7526.174004312543, 'time': 6.175606282541316, 'mode': 'drive'}
print edge attributes
{'osm_id': 447572464, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f00648e2990>, 'infra_type': 'residential', 'min_speed': 9.0, 'max_speed': 30.0, 'mean_speed': 23.486607142857142, 'length': 0.0177226656551865, 'speed': 23.486607142857142, 'i

print edge attributes
{'osm_id': 48652919, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009eea0550>, 'infra_type': 'secondary', 'min_speed': 18.0, 'max_speed': 49.0, 'mean_speed': 32.789186507936506, 'length': 0.026906393831197858, 'speed': 32.789186507936506, 'imp_cost': 5026.11436766776, 'time': 2.9541146978095028, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652919, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009eea0590>, 'infra_type': 'secondary', 'min_speed': 18.0, 'max_speed': 49.0, 'mean_speed': 32.789186507936506, 'length': 0.030903552527747125, 'speed': 32.789186507936506, 'imp_cost': 5772.783612183163, 'time': 3.3929719199639585, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652919, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009eea0510>, 'infra_type': 'secondary', 'min_speed': 18.0, 'max_speed': 49.0, 'mean_speed': 32.789186507936506, 'length': 0.08184770063567118, 'speed': 32.789186507936506, 'imp

print edge attributes
{'osm_id': 370806859, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f006c597090>, 'infra_type': 'secondary', 'min_speed': 22.0, 'max_speed': 55.0, 'mean_speed': 40.742559523809526, 'length': 0.19005428888168446, 'speed': 40.742559523809526, 'imp_cost': 35502.141163098655, 'time': 16.793138378413058, 'mode': 'drive'}
print edge attributes
{'osm_id': 370806859, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f006c597290>, 'infra_type': 'secondary', 'min_speed': 29.0, 'max_speed': 41.0, 'mean_speed': 34.59176587301587, 'length': 0.007915276024173728, 'speed': 34.59176587301587, 'imp_cost': 1478.5735613156523, 'time': 0.8237507674985051, 'mode': 'drive'}
print edge attributes
{'osm_id': 370806859, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f006c5ff790>, 'infra_type': 'secondary', 'min_speed': 29.0, 'max_speed': 41.0, 'mean_speed': 34.59176587301587, 'length': 0.015973508416037605, 'speed': 34.59176587301587, 'i

print edge attributes
{'osm_id': 370806859, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f006c597e50>, 'infra_type': 'secondary', 'min_speed': 15.0, 'max_speed': 43.0, 'mean_speed': 33.24355158730159, 'length': 0.03248108803627924, 'speed': 33.24355158730159, 'imp_cost': 6067.467245176962, 'time': 3.5174315422805504, 'mode': 'drive'}
print edge attributes
{'osm_id': 370806859, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f006c597f50>, 'infra_type': 'secondary', 'min_speed': 15.0, 'max_speed': 43.0, 'mean_speed': 33.24355158730159, 'length': 0.007549452600994752, 'speed': 33.24355158730159, 'imp_cost': 1410.2377458658198, 'time': 0.8175428937611648, 'mode': 'drive'}
print edge attributes
{'osm_id': 370806859, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f006c597fd0>, 'infra_type': 'secondary', 'min_speed': 25.0, 'max_speed': 48.0, 'mean_speed': 35.242063492063494, 'length': 0.01758511102887044, 'speed': 35.242063492063494, 'imp

print edge attributes
{'osm_id': 370806859, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f007a237c90>, 'infra_type': 'secondary', 'min_speed': 14.0, 'max_speed': 48.0, 'mean_speed': 32.42311507936508, 'length': 0.1443764676276529, 'speed': 32.42311507936508, 'imp_cost': 26969.524152845563, 'time': 16.030393199027824, 'mode': 'drive'}
print edge attributes
{'osm_id': 370806859, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009816bc10>, 'infra_type': 'secondary', 'min_speed': 23.0, 'max_speed': 23.0, 'mean_speed': 23.0, 'length': 0.010378554651602877, 'speed': 23.0, 'imp_cost': 1938.7140089194174, 'time': 1.6244694237291462, 'mode': 'drive'}
print edge attributes
{'osm_id': 370806859, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009816bd10>, 'infra_type': 'secondary', 'min_speed': 6.0, 'max_speed': 29.0, 'mean_speed': 17.703373015873016, 'length': 0.032994490068069815, 'speed': 17.703373015873016, 'imp_cost': 6163.370744715441,

...

print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f07f6d0>, 'infra_type': 'secondary', 'min_speed': 24.0, 'max_speed': 55.0, 'mean_speed': 43.80902777777778, 'length': 0.01852759051047721, 'speed': 43.80902777777778, 'imp_cost': 3460.9539073571427, 'time': 1.5225018499851606, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f07f710>, 'infra_type': 'secondary', 'min_speed': 24.0, 'max_speed': 55.0, 'mean_speed': 43.80902777777778, 'length': 0.013448328290217183, 'speed': 43.80902777777778, 'imp_cost': 2512.1477246125696, 'time': 1.105114272116761, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f07f690>, 'infra_type': 'secondary', 'min_speed': 24.0, 'max_speed': 55.0, 'mean_speed': 43.80902777777778, 'length': 0.01566607983709632, 'speed': 43.80902777777778, 'imp_cost

print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f017090>, 'infra_type': 'secondary', 'min_speed': 41.0, 'max_speed': 68.0, 'mean_speed': 53.398809523809526, 'length': 0.04119906582174261, 'speed': 53.398809523809526, 'imp_cost': 7695.98549550152, 'time': 2.777527032548203, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f017250>, 'infra_type': 'secondary', 'min_speed': 41.0, 'max_speed': 68.0, 'mean_speed': 53.398809523809526, 'length': 0.06850880711245085, 'speed': 53.398809523809526, 'imp_cost': 12797.445168605818, 'time': 4.618674232706528, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f017290>, 'infra_type': 'secondary', 'min_speed': 41.0, 'max_speed': 68.0, 'mean_speed': 53.398809523809526, 'length': 0.060390434589748875, 'speed': 53.398809523809526, 'imp_c

print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f017e10>, 'infra_type': 'secondary', 'min_speed': 35.0, 'max_speed': 66.0, 'mean_speed': 49.28373015873016, 'length': 0.035550191726533974, 'speed': 49.28373015873016, 'imp_cost': 6640.775814516546, 'time': 2.596814198181217, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f017ed0>, 'infra_type': 'secondary', 'min_speed': 35.0, 'max_speed': 66.0, 'mean_speed': 49.28373015873016, 'length': 0.035887652828400105, 'speed': 49.28373015873016, 'imp_cost': 6703.81354834514, 'time': 2.621464523203396, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f017f10>, 'infra_type': 'secondary', 'min_speed': 35.0, 'max_speed': 66.0, 'mean_speed': 49.28373015873016, 'length': 0.05073399716974989, 'speed': 49.28373015873016, 'imp_cost': 

print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f02cb50>, 'infra_type': 'secondary', 'min_speed': 5.0, 'max_speed': 58.0, 'mean_speed': 43.30952380952381, 'length': 0.028477281952487363, 'speed': 43.30952380952381, 'imp_cost': 5319.556268724639, 'time': 2.3671055696625007, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f02cc50>, 'infra_type': 'secondary', 'min_speed': 5.0, 'max_speed': 58.0, 'mean_speed': 43.30952380952381, 'length': 0.03551413849200294, 'speed': 43.30952380952381, 'imp_cost': 6634.041070306149, 'time': 2.9520273446898537, 'mode': 'drive'}
print edge attributes
{'osm_id': 48652498, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009f02cc90>, 'infra_type': 'secondary', 'min_speed': 5.0, 'max_speed': 58.0, 'mean_speed': 43.30952380952381, 'length': 0.06080099967624628, 'speed': 43.30952380952381, 'imp_cost': 1

print edge attributes
{'osm_id': 76752004, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009ba98c10>, 'infra_type': 'secondary', 'min_speed': 28.0, 'max_speed': 28.0, 'mean_speed': 28.0, 'length': 0.04696303950698375, 'speed': 28.0, 'imp_cost': 8772.695779904565, 'time': 6.038105079469339, 'mode': 'drive'}
print edge attributes
{'osm_id': 76752004, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009ba98cd0>, 'infra_type': 'secondary', 'min_speed': 28.0, 'max_speed': 28.0, 'mean_speed': 28.0, 'length': 0.1099505366784796, 'speed': 28.0, 'imp_cost': 20538.76025153999, 'time': 14.136497572947377, 'mode': 'drive'}
print edge attributes
{'osm_id': 76752004, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f009ba98d10>, 'infra_type': 'secondary', 'min_speed': 28.0, 'max_speed': 28.0, 'mean_speed': 28.0, 'length': 0.012557695526285403, 'speed': 28.0, 'imp_cost': 2345.777524310113, 'time': 1.614560853379552, 'mode': 'drive'}
print edge attr

In [ ]:
#import time

#start = time.time()

#results = {}

#count = 0
#for key in OD_df_dict:
    #while count > 1 and count < 6:
    #results[key] = generate_complete_edges_and_routes(OD_df_dict[key])
    #count += 1
    
#print(time.time() - start)

In [ ]:
#for key in results:
    # print edges
    #print(results[key][0])
    #file_name = f"./airport_output_edges_20200628/weighted_sec_saved_edges_{key}.shp"
    #print(file_name)
    #results[key][0].to_file(driver = 'ESRI Shapefile', filename = file_name)

In [ ]:
file_name = f"./airport_output_edges_20200628_traffic_mean/weighted_sec_saved_edges_3935302581.shp"
print(file_name)
results[0].to_file(driver = 'ESRI Shapefile', filename = file_name)

In [ ]:
#results[1243386867][0]

In [ ]:
# save as shapefile
#results[0].to_file(driver = 'ESRI Shapefile', filename = "./weighted_sec_saved_edges.shp")

In [ ]:
# save as shapefile
#results[1].to_file(driver = 'ESRI Shapefile', filename = "./shortest_path_routes.shp")